# Advent of code 2018

Lets start with necessary imports

In [1]:
import urllib.request as urllib2  # the lib that handles the url stuff
from datetime import datetime
from datetime import timedelta
import operator
from collections import Counter
import numpy as np
import scipy.misc as smp
import re
import string
from blist import blist

### Day 1

The first part requires us to get a sum of numbers found in a txt file.

In [2]:
fname = "day/1/input.txt"
with open(fname) as f:
    content = f.readlines()
day_1_content = [int(i) for i in content]
print("Sum of numbers is:")
print(sum(day_1_content))

Sum of numbers is:
406


In the second part, we have to keep summing the numbers found in the text file until we have found the first sum repeated twice

In [3]:
def find_reoccurring_frequency(number_list):
    not_found = True
    current_sum = 0
    sum_history = {}
    while(not_found):
        for number in number_list:
            current_sum += number
            if (current_sum in sum_history):
                print("First sum found twice is:")
                print(current_sum)
                not_found = False
                break
            else:
                sum_history[current_sum] = 1

find_reoccurring_frequency(day_1_content)

First sum found twice is:
312


### Day 2

The first part requires us to get find all entries of text blocks that have 2 or 3 of the same character. Multiplying the count of these gives the result.

In [4]:
fname = "day/2/input.txt"
with open(fname) as f:
    content = f.readlines()
day_2_content = [i.rstrip() for i in content]
#print(day_2_content)

def find_checksum(content):
    double = 0
    triple = 0
    for s in content:
        tmp = {}
        for letter in s:
            if letter in tmp:
                tmp[letter] += 1
            else:
                tmp[letter] = 1
        if (2 in tmp.values()):
            double += 1
        if (3 in tmp.values()):
            triple += 1
    print (double*triple)

def find_diff_of_one(content):
    for word1 in content:
        finished = False
        for word2 in content:
            differences = 0
            for i in range(len(word1)):
                if (word1[i]!=word2[i]):
                    differences+=1
            if (differences == 1):
                common = ""
                for i in range(len(word1)):
                    if (word1[i]==word2[i]):
                        common += word1[i]
                print(common)
                finished = True
                break
        if (finished):
            break

find_checksum(day_2_content)
find_diff_of_one(day_2_content)

5390
nvosmkcdtdbfhyxsphzgraljq


### Day 3

In [5]:
fname = "day/3/input.txt"
with open(fname) as f:
    content = f.readlines()
day_3_content = [x.rstrip() for x in content]
#print(content)
checklist = [0] * 1000000
for entry in day_3_content:
    [startx, starty] = [int(i) for i in entry.split('@')[1].split(':')[0].split(',')]
    [lenx, leny] = [int(i) for i in entry.split('@')[1].split(':')[1].split('x')]
    for i in range(startx, startx+lenx):
        for j in range(starty, starty+leny):
            checklist[i+1000*j]+=1

#print(checklist)
print(sum(i > 1 for i in checklist))

for entry in day_3_content:
    [startx, starty] = [int(i) for i in entry.split('@')[1].split(':')[0].split(',')]
    [lenx, leny] = [int(i) for i in entry.split('@')[1].split(':')[1].split('x')]
    entryid = entry.split('@')[0].split('#')[1]
    theone = True
    for i in range(startx, startx+lenx):
        for j in range(starty, starty+leny):
            if(checklist[i+1000*j]!=1):
                theone = False
                break
        if (not theone):
            break
    if (theone):
        print(entryid)
        break

118322
1178 


### Day 4


In [6]:
fname = "day/4/input.txt"
with open(fname) as f:
    content = f.readlines()
day_4_content = [i.rstrip() for i in content]


def get_sorted_entries(content):
    entrylist = []
    for entrystring in day_4_content:
        date = datetime.strptime(entrystring.split(']')[0][1:], '%Y-%m-%d %H:%M')
        entrylist.append({"time":date, "msg" : entrystring.split(']')[1]})
    sortedentries = sorted(entrylist, key=lambda r: r['time'])
    return sortedentries

def generate_guard_history(sortedentries):
    guard_history = {}
    guard = ""
    sleepytime = 0
    for entry in sortedentries:
        if "Guard" in entry['msg']:
            guard = entry['msg'].split('#')[1].split(' ')[0]
        if "falls asleep" in entry['msg']:
            sleepytime = entry['time']
        if "wakes" in entry['msg']:
            if not guard in guard_history:
                guard_history[guard] = {"total":0, "times":{}}
            while sleepytime < entry['time']:
                guard_history[guard]["total"] += 1
                clock = sleepytime.strftime("%M")
                if (clock in guard_history[guard]["times"]):
                    guard_history[guard]["times"][clock] +=1
                else:
                    guard_history[guard]["times"][clock]=0
                sleepytime += timedelta(minutes=1)
    return guard_history

def get_max_sleeper_times_max_minute(guard_history):
    maxim = 0
    maxguard = ""
    for guard in guard_history.items():
        if guard[1]['total'] > maxim:
            maxim = guard[1]['total']
            maxguard = guard[0]
    maxtimes = guard_history[maxguard]["times"]
    max_key = max(maxtimes, key=lambda k: maxtimes[k])
    return int(maxguard)*int(max_key)

def get_most_frequent_sleep_minute_times_sleeper(guard_history):
    maxim = 0
    maxmin = 0
    maxguard = 0
    for guard in guard_history.items():
        for time in guard[1]["times"].items():
            if (time[1] > maxim):
                maxim = time[1]
                maxguard = guard[0]
                maxmin = time[0]
    return int(maxmin)*int(maxguard)        

sortedentries = get_sorted_entries(day_4_content)
guard_history = generate_guard_history(sortedentries)
part1_ans = get_max_sleeper_times_max_minute(guard_history)
print(part1_ans)
part2_ans = get_most_frequent_sleep_minute_times_sleeper(guard_history)
print(part2_ans)




    

72925
49137


### Day 5

In [7]:
fname = "day/5/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_5_content = content[0].rstrip()

alphabet = "abcdefghijklmnopqrstuvwxyz"
reactable = [c + c.upper() for c in alphabet]
reactable += [c.upper() + c for c in alphabet]

reacting = True
def reaction_start(polymer):
    while(True):
        old = polymer
        for pair in reactable:
            polymer = polymer.replace(pair, '')
        if (polymer == old):
            break
    return polymer
print(len(reaction_start(day_5_content)))


results = []
for c in alphabet:
    test_str = day_5_content.replace(c, '')    
    test_str = test_str.replace(c.upper(), '')
    results.append((c, len(reaction_start(test_str))))
print(min(results, key = lambda t: t[1]))

10886
('v', 4684)


### Day 6


In [8]:
def manhattan_dist(x, x2, y, y2):
    return (abs(x-x2)+abs(y-y2))

def visualizer(array, x, y, name):
    with open("day/6/"+name, "w") as the_file:
        for i in range(x):
            line = ""
            for j in range(y):
                dist = array[i,j]["dist"]
                if dist==0:
                    line+="0"
                else:
                    line+=(array[i,j]["closest"])
            line+=("\n")
            the_file.write(line)
        
def draw_image(array, x, y, identifiers):
    colors = {}
    for idt in identifiers:
        color = list(np.random.choice(range(256), size=3))
        colors[idt] = color
    colors['.'] = [255, 255, 255]

    data = np.zeros( (x, y, 3), dtype=np.uint8 )
    for i in range(x):
        for j in range(y):
            data[i,j] = colors[array[i,j]["closest"]]
    img = smp.toimage(data)
    img.show()

fname = "day/6/input.txt"
with open(fname) as f:
    content = f.readlines()
day_6_content = [i.rstrip().split(', ') for i in content]
day_6_content = [[int(i[0]), int(i[1])]  for i in day_6_content]
x_len = max(day_6_content, key=lambda item:item[0])[0]
y_len = max(day_6_content, key=lambda item:item[1])[1]
#print(x_len)
#print(y_len)
#print(day_6_content)
array = {}
for i in range(x_len):
    for j in range(y_len):
        array[i, j] = {"closest":"", "dist":360}

identifiers = "abcdefghijklmnopqrstuvwxyABCDEFGHIJKLMNOPQRSTUVWXY"
        
for idx, coord in enumerate(day_6_content):
    x = coord[0]
    y = coord[1]
    for i in range(x_len):
        for j in range(y_len):
            dist = manhattan_dist(x, i, y, j)
            if (dist<array[i,j]["dist"]):
                array[i,j]["dist"] = dist
                array[i,j]["closest"] = identifiers[idx]
            elif (dist==array[i,j]["dist"]):
                array[i,j]["closest"] = '.'


visualizer(array, x_len, y_len, 'test_print.txt')
draw_image(array, x_len, y_len, identifiers)
infinite = []
for i in range(x_len):
    a = array[i, 0]["closest"]
    b = array[i, y_len-1]["closest"]
    if a not in infinite:
        infinite.append(a)
    if b not in infinite:
        infinite.append(b)
for j in range(y_len):
    a = array[0, j]["closest"]
    b = array[x_len -1, j]["closest"]
    if a not in infinite:
        infinite.append(a)
    if b not in infinite:
        infinite.append(b)

#print(infinite)
counts = Counter()
for i in range(x_len):
    for j in range(y_len):
        val = array[i, j]["closest"]
        if val not in infinite:
            counts.update(str(val))
maxim = counts.most_common()[0]
print(maxim)

safe_array = {}
for i in range(x_len):
    for j in range(y_len):
        safe_array[i, j] = {"closest":"#", "dist":10000}
for idx, coord in enumerate(day_6_content):
    x = coord[0]
    y = coord[1]
    for i in range(x_len):
        for j in range(y_len):
            dist = manhattan_dist(x, i, y, j)
            if (safe_array[i,j]['closest']=='#'):
                safe_array[i,j]['dist']-=dist
                if(safe_array[i,j]['dist']<=0):
                    safe_array[i,j]['closest']='.'
visualizer(safe_array, x_len, y_len, 'test_print2.txt')
safecount = 0
for i in range(x_len):
    for j in range(y_len):
        if safe_array[i,j]['closest']=='#':
            safecount+=1
print(safecount)
draw_image(safe_array, x_len, y_len, "#.")

/home/paavo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


('V', 5975)
38670


### Day 7


In [9]:
fname = "day/7/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_7_content = [i.rstrip() for i in content]
Alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
ruleset = {}
for letter in Alphabet:
    ruleset[letter] = {"done":False, "requires":[]}
for rule in day_7_content:
    regex = re.search(r'tep (\w)(.*)tep (\w)', rule)
    requirement = regex.group(1)
    letter = regex.group(3)
    ruleset[letter]["requires"].append(requirement)

def not_ready_yet(ruleset):
    for key in ruleset:
        if not ruleset[key]["done"]:
            return True
    return False
ready = []
while(not_ready_yet(ruleset)):
    for letter in Alphabet:
        if (not ruleset[letter]["done"]):
            if (set(ruleset[letter]["requires"]).issubset(ready)):
                ready.append(letter)
                ruleset[letter]["done"] = True
                break
print(''.join(ready))
    
class Workers:
    def __init__(self, amount):
        self.available = amount
        self.at_work = 0
        self.jobs = []
        self.time_spent = 0
        self.completed = 0
        
    def start_job(self, letter):
        if (self.available == 0):
            return False
        time = string.ascii_uppercase.index(letter) + 61
        self.jobs.append({"letter":letter, "time":time})
        self.available -=1
        self.at_work += 1
        return True
    
    def spend_second(self):
        done = []
        for job in self.jobs:
            job["time"] -= 1
            if (job["time"] == 0):
                done.append(job["letter"])
                self.completed += 1
                self.at_work -= 1
                self.available += 1
        self.time_spent += 1
        return done
    
    def time(self):
        return self.time_spent
    
    def done(self):
        return (self.completed==26)
    
workers = Workers(5)

ruleset = {}
for letter in Alphabet:
    ruleset[letter] = {"done":False, "requires":[]}
for rule in day_7_content:
    regex = re.search(r'tep (\w)(.*)tep (\w)', rule)
    requirement = regex.group(1)
    letter = regex.group(3)
    ruleset[letter]["requires"].append(requirement)

ready = []
while(not workers.done()):
    for letter in Alphabet:
        if (not ruleset[letter]["done"]):
            if (set(ruleset[letter]["requires"]).issubset(ready)):
                if(workers.start_job(letter)):
                    print("starting", letter)
                    ruleset[letter]["done"] = True
                else:
                    break

    new = workers.spend_second()
    for letter in new:
        ready.append(letter)

print(workers.time())

AEMNPOJWISZCDFUKBXQTHVLGRY
starting A
starting E
starting M
starting P
starting N
starting O
starting J
starting W
starting I
starting Z
starting S
starting C
starting D
starting F
starting U
starting X
starting K
starting Q
starting B
starting T
starting H
starting V
starting L
starting G
starting R
starting Y
1081


### Day 8

In [10]:
fname = "day/8/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_8_content = content[0].rstrip().split(" ")
day_8_content = [int(i) for i in day_8_content]

class Node:
    def __init__(self):
        self.entries = []
        self.children = []
            
    def populate(self, numlist):
        childamount = numlist[0]
        entryamount = numlist[1]
        numlist = numlist[2:]
        while(childamount>0):
            child = Node()
            numlist = child.populate(numlist)
            self.children.append(child)
            childamount -= 1
        self.entries = numlist[:entryamount]
        numlist = numlist[entryamount:]
        return numlist
    
    def entrysum(self):
        entrysum = sum(self.entries)
        for child in self.children:
            entrysum += child.entrysum()
        return entrysum
    
    def nodevalue(self):
        value = 0
        childamount = len(self.children)
        if(childamount==0):
            value += sum(self.entries)
        else:
            for entry in self.entries:
                if (entry <= childamount and entry!=0):
                    value += self.children[entry-1].nodevalue()
        return value

rootnode = Node()
rootnode.populate(day_8_content)
print(rootnode.entrysum())
print(rootnode.nodevalue())

38722
13935


### Day 9

In [11]:
fname = "day/9/input.txt"
with open(fname) as f:
    content = f.readlines()[0].rstrip()
game_setting = [int(s) for s in content.split() if s.isdigit()]
players = game_setting[0]
marbles = game_setting[1]

#players = 17
#marbles = 1104

def solve_game(players, marbles):
    table = blist([0])
    player = 1
    current = 0
    gamecount = [0] * players
    for i in range(1, marbles+1):
        #print(i)
        #print(player)
        if (i % 23 == 0):
            gamecount[player-1]+=i
            index = current
            for j in range(7):
                index-=1
                if(index<1):
                    index = len(table)
            gamecount[player-1]+=table[index]
            table.pop(index)
            current = index
        else:
            index = current
            for j in range(2):
                index+=1
                if index>len(table):
                    index = 1
            table.insert(index, i)
            current = index

        #print(table)
        #print(table[current])

        player += 1
        if (player > players):
            player = 1
    print(max(gamecount))
solve_game(players,marbles)
solve_game(players,100*marbles)

390093
3150377341


### Day 10

In [12]:
class MessagePoint:
    def __init__(self, x, y, xspeed, yspeed):
        self.x = x
        self.y = y
        self.xspeed = xspeed
        self.yspeed = yspeed
        self.time = 0
        self.limit = 235
    
    def moveSecond(self):
        self.time+=1
        self.x+=self.xspeed
        self.y+=self.yspeed

    def close_enough(self):
        return (abs(self.x)<self.limit and abs(self.y)<self.limit)
    
    def get_limit(self):
        return self.limit

    def get_coords(self):
        newx = self.x + self.limit
        newy = self.y + self.limit
        return (newx, newy)
    
    def get_time(self):
        return self.time

def draw_message(points):   
    limit = points[0].get_limit()
    data = np.zeros( (2*limit, 2*limit, 3), dtype=np.uint8 )
    for point in points:
        (i, j) = point.get_coords()
        data[j,i] = [255,255,255]
    img = smp.toimage(data)
    img.show()    

fname = "day/10/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_10_content = [i.rstrip() for i in content]
pointlist = []
for point in day_10_content:
    coords = [int(s) for s in re.findall(r'-?\d+\.?\d*', point)]
    pointlist.append(MessagePoint(coords[0], coords[1], coords[2], coords[3]))
print(len(pointlist))

found = False
while(not found):
    found = True
    for point in pointlist:
        if not point.close_enough():
            found = False
            break
    for point in pointlist:
        point.moveSecond()
print("asd")
draw_message(pointlist)
print(pointlist[0].get_time())

352
asd
10391


/home/paavo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:35: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.


### Day 11

In [13]:
grid_serial = 8444
grid = {}
for x in range(1,301):
    for y in range(1,301):
        rack_id = x + 10
        power = rack_id * y
        power += grid_serial
        power = power*rack_id
        str_power = str(power)
        power = int(str_power[-3])
        power -= 5
        grid[x,y] = power
maximum = -999
maxcoord = (-1, -1)
for x in range(1,298):
    for y in range(1,298):
        total = grid[x,y]+grid[x+1,y]+grid[x+1,y+1]+grid[x,y+1]+grid[x+1,y+2]+grid[x+2,y+1]+grid[x+2,y]+grid[x,y+2]+grid[x+2,y+2]
        if (total > maximum):
            maximum = total
            maxcoord = (x,y)
print(maximum)
print(maxcoord)


class WindowSum:
    def __init__(self, size, arraysize, array):
        self.array = array
        self.size = size
        self.arraysize = arraysize
        self.positionx = 1
        self.positiony = 1
        self.sum = 0
        for xsize in range(size):
            for ysize in range(size):
                self.sum+=self.array[self.positionx+xsize,self.positiony+ysize]
        self.direction = "right"
    
    def moveStep(self):
        if self.direction == "right":
            if(self.positionx+size == self.arraysize):
                if not(self.movedown()):
                    return False
                self.direction = "left"
            else:
                self.moveright()
        elif self.direction == "left":
            if(self.positionx == 1):
                if not(self.movedown()):
                    return False
                self.direction = "right"
            else:
                self.moveleft()
        return True
    
    def movedown(self):
        if(self.positiony+self.size==self.arraysize):
            return False
        for i in range(size):
            self.sum -= self.array[self.positionx+i, self.positiony]
        self.positiony+=1
        for i in range(size):
            self.sum += self.array[self.positionx+i, self.positiony+self.size-1]
        return True
    
    def moveright(self):
        for i in range(size):
            #print("removing", self.array[self.positionx, self.positiony+i])
            self.sum -= self.array[self.positionx, self.positiony+i]
        self.positionx+=1
        for i in range(size):
            #print("adding", self.array[self.positionx+self.size-1, self.positiony+i])
            self.sum += self.array[self.positionx+self.size-1, self.positiony+i]
    
    def moveleft(self):
        for i in range(size):
            self.sum -= self.array[self.positionx+self.size-1, self.positiony+i]
        self.positionx-=1
        for i in range(size):
            self.sum += self.array[self.positionx, self.positiony+i]
    
    def getSum(self):
        return self.sum
    
    def getcoord(self):
        return (self.positionx, self.positiony, self.size)
        

maxcoord = (1, 1, 3)
maximum = -999
## Takes a long time to run, optimal answer found on window size 12
for size in range(1, 13):
    window = WindowSum(size, 300, grid)
    while (window.moveStep()):
        wsum = window.getSum()
        #print(window.getcoord())
        if wsum > maximum:
            maximum = wsum
            maxcoord = window.getcoord()
print(maximum)
print(maxcoord)

28
(243, 68)
96
(236, 252, 12)


### Day 12

In [14]:
fname = "day/12/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_12_content = [i.rstrip() for i in content]
recipes = {}
for i in range(2, len(day_12_content)):
    recipe = day_12_content[i].split(" => ")
    recipes[recipe[0]] = recipe[1]
state = []
for char in day_12_content[0].split(": ")[1]:
    state.append(char)
    
def simulate_generations(generations, padding_left, padding_right, state):
    state = ['.'] * padding_left + state
    state+=['.'] * padding_right


    #print("0 :", ''.join(state))
    for generation in range(generations):
        new_state = state.copy()
        for i in range(2, len(state)-2):
            recipe = ''.join(state[i-2:i+3])
            if recipe in recipes:
                new_state[i] = recipes[recipe]
        state = new_state
        #print(generation+1,":", ''.join(state))
    #print(new_state.count('#'))
    potsum = 0
    testlist = []
    for pot in range(len(state)):
        potnum = pot-padding_left
        if (state[pot]=='#'):
            testlist.append(potnum-generations)
            potsum += potnum
    print(potsum)
    return testlist
simulate_generations(20, 5, 28, state)


simulate_generations(100, 5, 101, state)

# Finding a pattern....

a = simulate_generations(103, 5, 104, state)
b = simulate_generations(153, 5, 155, state)
c = simulate_generations(200, 5, 220, state)

print(a)
print(b)
print(c)

newsum = 0
for i in c:
    newsum += (50000000000+i)
print(newsum)









2840
5485
5602
7804
9684
[-19, -16, -15, -13, -8, -5, -4, -2, 7, 10, 11, 13, 18, 23, 28, 33, 36, 37, 39, 44, 47, 48, 50, 55, 60, 63, 64, 66, 71, 74, 75, 77, 82, 85, 86, 88, 91, 92, 94]
[-19, -16, -15, -13, -8, -5, -4, -2, 7, 10, 11, 13, 18, 23, 28, 33, 36, 37, 39, 44, 47, 48, 50, 55, 60, 63, 64, 66, 71, 74, 75, 77, 82, 85, 86, 88, 91, 92, 94, 99]
[-19, -16, -15, -13, -8, -5, -4, -2, 7, 10, 11, 13, 18, 23, 28, 33, 36, 37, 39, 44, 47, 48, 50, 55, 60, 63, 64, 66, 71, 74, 75, 77, 82, 85, 86, 88, 91, 92, 94, 99]
2000000001684


In [15]:
### Day 13


In [16]:
fname = "day/13/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_13_content = [i.rstrip('\n') for i in content]
#print(day_13_content)
roadmap = {}
height = len(day_13_content)
width = len(day_13_content[0])
class Cart:
    def __init__(self, x, y, direction, cartid):
        self.x = x
        self.y = y
        self.direction = direction
        self.hides =  " "
        self.collision = False
        self.nextturn = "left"
        self.id = cartid
        self.moved = False

    def move(self, roadmap):
        revealed = self.hides
        if(self.direction=='<'):
            self.x -= 1
        elif(self.direction=='>'):
            self.x += 1
        elif(self.direction=='^'):
            self.y -= 1
        elif(self.direction=='v'):
            self.y += 1

        self.hides = ' '

        if(roadmap[self.x, self.y]=='\\'):
            if(self.direction=='<' or self.direction=='>'):
                self.turnRight()
            else:
                self.turnLeft()
            self.hides = '\\'
        elif(roadmap[self.x, self.y]=='/'):
            if(self.direction=='<' or self.direction=='>'):
                self.turnLeft()
            else:
                self.turnRight()
            self.hides = '/'
        elif(roadmap[self.x, self.y]=='+'):
            if self.nextturn=='left':
                self.turnLeft()

                self.nextturn='straight'
            elif self.nextturn=='straight':
                self.nextturn='right'
            elif self.nextturn=='right':
                self.turnRight()
                self.nextturn='left'
            self.hides = '+'
        elif(isinstance(roadmap[self.x, self.y], Cart)):
            self.collision = True
            self.hides = roadmap[self.x, self.y].hides
        else:
            self.hides = roadmap[self.x, self.y]
        self.moved = True
        return revealed, self.x, self.y
                
    def turnRight(self):
        if(self.direction=='<'):
            self.direction='^'
        elif(self.direction=='>'):
            self.direction='v'
        elif(self.direction=='v'):
            self.direction='<'
        elif(self.direction=='^'):
            self.direction='>'
    
    def turnLeft(self):
        if(self.direction=='<'):
            self.direction='v'
        elif(self.direction=='>'):
            self.direction='^'
        elif(self.direction=='v'):
            self.direction='>'
        elif(self.direction=='^'):
            self.direction='<'
            
    def collision(delf):
        return self.collision
    
    def __str__(self):
        return self.direction
cartid = 0
cartlist = []
for x in range(width):
    for y in range(height):
        mark = day_13_content[y][x]
        if (mark=='<' or mark=='>' or mark=='^' or mark=='v'):
            roadmap[x,y] = Cart(x,y,mark,cartid)
            cartlist.append(roadmap[x,y])
            cartid += 1
        else:
            roadmap[x,y] = mark
finished = False
while(len(cartlist)!=1):
    for y in range(height):
        for x in range(width):
            if isinstance(roadmap[x,y], Cart):
                if not roadmap[x,y].moved:
                    revealed, newx, newy = roadmap[x,y].move(roadmap)
                    tmp = roadmap[newx, newy]
                    roadmap[newx,newy] = roadmap[x,y]
                    roadmap[x,y] = revealed

                    if roadmap[newx,newy].collision:
                        collision = True
                        print("Crash happened in", roadmap[newx,newy].x, roadmap[newx,newy].y)
                        cartlist.remove(tmp)
                        cartlist.remove(roadmap[newx, newy])
                        roadmap[newx, newy] = roadmap[newx, newy].hides
                        break
    if not finished:
        for cart in cartlist:
            cart.moved = False
print("Final cart found in...", cartlist[0].x, cartlist[0].y)


Crash happened in 8 3
Crash happened in 104 65
Crash happened in 83 71
Crash happened in 103 137
Crash happened in 122 111
Crash happened in 53 77
Crash happened in 111 61
Crash happened in 61 31
Final cart found in... 73 121


### Day 14

In [17]:
day_14_content = int("074501")
scores = [3, 7]
elf1 = 0
elf2 = 1
while(len(scores) < day_14_content+10):
    #print(elf1, elf2, scores)
    elf1score = scores[elf1]
    elf2score = scores[elf2]
    new = str(elf1score + elf2score)
    for num in new:
        scores.append(int(num))
    elf1 += 1 + elf1score
    elf2 += 1 + elf2score
    scorelen = len(scores)
    if elf1 >= scorelen:
        multiplier = int(elf1/scorelen)
        elf1 -= multiplier*scorelen
    if elf2 >= scorelen:
        multiplier = int(elf2/scorelen)
        elf2 -= multiplier*scorelen
print(''.join(str(x) for x in scores[-10:]))

for i in range(25000000):
    #print(elf1, elf2, scores)
    elf1score = scores[elf1]
    elf2score = scores[elf2]
    new = str(elf1score + elf2score)
    for num in new:
        scores.append(int(num))
    elf1 += 1 + elf1score
    elf2 += 1 + elf2score
    scorelen = len(scores)
    if elf1 >= scorelen:
        multiplier = int(elf1/scorelen)
        elf1 -= multiplier*scorelen
    if elf2 >= scorelen:
        multiplier = int(elf2/scorelen)
        elf2 -= multiplier*scorelen
#print(''.join(str(x) for x in scores))
print(''.join(str(x) for x in scores).find("074501"))


1464411010
20288091


### Day 15

In [18]:
class Unit:
    def __init__(self, x, y, mark, unitid, strength):
        self.x = x
        self.y = y
        self.type = mark
        self.id = unitid
        self.moved = False
        self.hp = 200
        self.strength = strength
        
    def act(self, field, unitlist):
        # Check if enemies next to unit
        if self.moved:
            return (self.x, self.y)
        # Get targets
        foes = [unit for unit in unitlist if unit.type!=self.type]
        
        # Check if any foe is in attack distance
        nearfoe = None
        #print("Possibilities")
        for foe in foes:
            manhattan = abs(foe.x-self.x)+abs(foe.y-self.y)
            if (manhattan==1):
                #print(foe.x, foe.y, foe.hp)
                if (not nearfoe):
                    nearfoe = foe
                else:
                    if (foe.hp<nearfoe.hp):
                        nearfoe = foe
                    elif (foe.hp==nearfoe.hp and (foe.y<nearfoe.y or (foe.y==nearfoe.y and foe.x<nearfoe.x))):
                        nearfoe = foe
        #if nearfoe:
        #    print("chosen",nearfoe.x, nearfoe.y,nearfoe.hp)
        #else:
        #    print("none chosen")
        #print("--------")
        if nearfoe:
            #print("Near:", nearfoe.x, nearfoe.y)
            self.attack(nearfoe, field, unitlist)
            return (self.x, self.y)
            
        # Get attack positions
        attack_positions = []
        for foe in foes:
            for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
                if(field[foe.x+i, foe.y+j]=='.'):
                    attack_positions.append((foe.x+i, foe.y+j))
        # Get targetspot
        #print(self.x, self.y)
        #print(attack_positions)
        target = closest(field, (self.x, self.y), attack_positions)[0]
        #print(target)
        closest1 = 999
        selected =(-1, -1)
        for i, j in ((-1, 0), (0, -1), (0, 1), (1, 0)):
            coord = (self.x+i, self.y+j)
            if(field[coord]=='.'):
                dist = closest(field, coord, [target])[1]
                if dist<closest1:
                    selected = coord
                    closest1 = dist
                elif dist == closest1:
                    if (coord[1]<selected[1] or (coord[1]==selected[1] and coord[0]<selected[0])):
                        selected = coord
        #print(selected)
        self.moved = True
        if selected==(-1, -1):
            return (self.x, self.y)
        self.move(selected)
        # Check if any foe is in attack distance
        nearfoe = None
        #print("Possibilities")
        for foe in foes:
            manhattan = abs(foe.x-self.x)+abs(foe.y-self.y)
            if (manhattan==1):
                #print(foe.x, foe.y, foe.hp)
                if (not nearfoe):
                    nearfoe = foe
                else:
                    if (foe.hp<nearfoe.hp):
                        nearfoe = foe
                    elif (foe.hp==nearfoe.hp and (foe.y<nearfoe.y or (foe.y==nearfoe.y and foe.x<nearfoe.x))):
                        nearfoe = foe
        #if nearfoe:
        #    print("chosen",nearfoe.x, nearfoe.y,nearfoe.hp)
        #else:
        #    print("none chosen")
        #print("--------")
        if nearfoe:
            #print("Near:", nearfoe.x, nearfoe.y)
            self.attack(nearfoe, field, unitlist)
            return (self.x, self.y)
        return selected
                        
    def move(self, coord):
        self.x = coord[0]
        self.y = coord[1]
        
    def attack(self, foe, field, unitlist):
        if self.type == 'E':
            damage = self.strength
        else:
            damage = 3
        foe.hp -= damage
        if foe.hp<1:
            field[foe.x, foe.y] = "."
            unitlist.remove(foe)
    
    def __repr__(self):
        return self.type

def closest(grid, start, goals):
    done = []
    todo = [start]
    distances = {}
    distance = 0
    while set(done)!=set(todo):
        tmp = todo.copy()
        for coord in tmp:
            if coord not in done:
                done.append(coord)
                if coord not in distances:
                    distances[coord] = distance
                if (grid[coord[0]-1, coord[1]]=='.'):
                    todo.append((coord[0]-1, coord[1]))
                if (grid[coord[0]+1, coord[1]]=='.'):
                    todo.append((coord[0]+1, coord[1]))
                if (grid[coord[0], coord[1]-1]=='.'):
                    todo.append((coord[0], coord[1]-1))
                if (grid[coord[0], coord[1]+1]=='.'):
                    todo.append((coord[0], coord[1]+1))
        distance += 1
    closest = 999
    selected =(-1, -1)
    for goal in goals:
        dist = distances.get(goal, 999)
        if dist<closest:
            selected = goal
            closest = dist
        elif dist == closest:
            if (goal[1]<selected[1] or (goal[1]==selected[1] and goal[0]<selected[0])):
                selected = goal
                
    #print(selected, closest)
    return selected, closest


def gameover(unitlist):
    types = []
    for unit in unitlist:
        if unit.type not in types:
            types.append(unit.type)
    return len(types)==1

fname = "day/15/input.txt"
content = []
with open(fname) as f:
    content = f.readlines()
day_15_content = [i.rstrip('\n') for i in content]
#print(day_13_content)
height = len(day_15_content)
width = len(day_15_content[0])

def run_simulation(day_15_content, height, width, strength):
    field = {}
    unitid = 0
    unitlist = []
    for x in range(width):
        for y in range(height):
            mark = day_15_content[y][x]
            if (mark=='E' or mark=='G'):
                field[x,y] = Unit(x,y,mark,unitid,strength)
                unitlist.append(field[x,y])
                unitid += 1
            else:
                field[x,y] = mark
    stopgame = False
    i = 0
    while not stopgame:
        #print("Round", i+1)

        for y in range(height):
            for x in range(width):
                tile = field[x,y]
                if isinstance(tile, Unit):
                    #print(tile)
                    move = tile.act(field, unitlist)
                    #print(move)
                    if move!=(x,y):
                        field[move] = tile
                        field[x,y] = "."
                    else:
                        stopgame = gameover(unitlist)
                    #print("End")
        if(not stopgame):
            i+=1
        for unit in unitlist:
            unit.moved = False

    #for y in range(height):
    #    string = ""
    #    for x in range(width):
    #        string+=(str(field[x,y]))
    #    print(string)

    #for unit in unitlist:
    #    print(unit.type, unit.hp)
    print("---------")
    print("strength", strength)
    print("Game ends at turn", i)
    totalhp = 0
    elves = 0
    gnomes = 0
    for unit in unitlist:
        totalhp += unit.hp
        if unit.type == 'E':
            elves+=1
        else:
            gnomes+=1
    print("Remaining HP", totalhp)
    print("Outcome", i*totalhp)
    print("Elves alive", elves, ", Gnomes alive", gnomes)
    return elves
run_simulation(day_15_content, height, width, 3)
run_simulation(day_15_content, height, width, 25)


---------
strength 3
Game ends at turn 63
Remaining HP 3000
Outcome 189000
Elves alive 0 , Gnomes alive 18
---------
strength 25
Game ends at turn 29
Remaining HP 1328
Outcome 38512
Elves alive 10 , Gnomes alive 0


10